# Reducing Complexity (Optimization)

- We need to start reducing the complexity from the training samples
  - Some bi-grams won't add any value in learning context
    - E.g., $\texttt{(of, the)}$, $\texttt{(returns, the)}$ won't contribute much in finding the relationship between the words $\texttt{happy}$ and $\texttt{returns}$
  - Similarly in tri-grams it will exist
  - Normally bigrams and tri-grams won't be used in the training, instead we will go with __*5-words window*__

## Sub-Sampling

- Removing following kind of words from the training samples, the resultant sub-samples are used for training
  - Words pairs that does not give much informatino
  - Words pairs that appear in switched order
  - Less frequent words

- The words $\texttt{(of, the)}$ in the pairs $\texttt{(of, happy)}$, $\texttt{(returns, the)}$ do __*not give much information*__ about the words happy and returns, respectively. Similarly some pairs __*reappear*__ with the __*order of the words switched*__
  - The pair $\texttt{(of, the)}$ does not add any value information
  - The pairs $\texttt{(wish, you)}$, $\texttt{(you, wish)}$ can be reduced to one pair
- Some word could also be __*randomly removed from the based on the frequencies*__
  - Support a word appeared only once in the corpus, that won't give any information to the model
    - This word can be removed from the training samples
- Words with _less frequency or infrequent_ __*words appearing as context words*__ _could be discarded as they_ __*may not provide contextual information to the central word*__

### Sub-Sampling in Google's Word2Vec.c

- Here is the code for sub-sampling used by [word2vec.c](https://github.com/tmikolov/word2vec/blob/master/word2vec.c) that randomly removes a word from the sample

```c
        if (word == 0) break;
        // The subsampling randomly discards frequent words while keeping the ranking same
        if (sample > 0) {
          real ran = (sqrt(vocab[word].cn / (sample * train_words)) + 1) * (sample * train_words) / vocab[word].cn;
          next_random = next_random * (unsigned long long)25214903917 + 11;
          if (ran < (next_random & 0xFFFF) / (real)65536) continue;
        }
```

- Let $\displaystyle \large f(X) = \frac{vocab[word].cn}{train \_ words}$ and $\displaystyle \large ran = (\sqrt{f(x)} + 1) \times \frac{1}{f(x)}$
  - where
    - ${vocab[word].cn}$ is the count of the word
    - ${train \_ words}$ represents all the training words
  - Then, the probability of keeping the word is decided based on the generated random value random
- If $ran \lt random$ keep it, else discard the word

## Negative Sampling

- __*Negative-Sampling:*__
  - Another mechanism to minimize the computation
  - The size of the network is proportional to the size of the vocabulary $\mathbf{V}$. For every training cycle of input, the every weight in the network needs to be updated
  - For every training cycle, Softmax function computes the sum of the output neuron values
  - Cost of updating all the weights in the fully connected network is very high
  - Is it possible to change only a small percentage of the weights?
- __*Intuition behind negative sampling*__
  - In case of Skip-Gram model, the learning happens between the context words and the input word
    - The error that occurs doesn't have impact on other words that are in the training.
    - Other than input and context words, rest of the words doesn't contribute anything to the learning in that iteration
  - Can we really curtail the updation of the weights by only choosing the certain number of weights to be updated
    - Why should I update all my weights in every iteration, when it does n't make any impact on the learning
      - Computation of weights in each iteration is costly
    - Which are the weights that we should not bother about
- __*How Negative Sampling is performed*__
  - Select a small number of _negative_ words
    - Negative words are words that are not the context words
    - __*These words are choosen by soem mechasnim*__
  - While updating the weights, these samples output zero while the positive sample(s) will retainits value
  - During the backpropagation, the weights related to the negative and positive words are changed and the rest will remain untouched for the current update
  - This reduces drastically the computation

### Selecting a Negative Sample

- Several mechanism available in selecting negative words
- Below is one kind of mechanism
  1. Pick the most frequently occuring words as negative words
    - Given the probability of a given word in the corpus $\displaystyle \large P(w_i) = \frac{f(w_i)}{\sum_{{j=0}}^{n} f(w_j)}$
    - Apply some mechanism ($\frac{3}{4}$), which boosts the probability of less frequent words and reduces the probability of highly appearing words in the corpus
  2. Once we have the probability of words, find the number of occurence of that word in the entire corpus (say using Frequency Table)
    - Using Frequency Table, find out how many times you want to repeat certain words in a table
      - E.g., create a unigram table of size of the vocabulary
      - Pickup the most frequently appearing word and create another unigram table
      - Use the probability (above formula) to pick words which you want to use as negative sample words

![Word2Vec_Selectin_A_Negative_Sample](images/Word2Vec_Selectin_A_Negative_Sample.jpg)

## Trouble with the size of the network

- All weights ($output \rightarrow hidden$) and ($hidden \rightarrow input$) are adjusted by taking a training sample so that the prediction cycle minimizes the loss function
- This amounts to updating all the weights in the neural network
  - amounts to several million weights for a network which has input neurons, $|V| = 1M$, and hidden unit size as 300
- In addition, we should consider the seveeral million training samples pairs

## Softmax

- As part of reducing the complexity of Word2Vec learning, optimization can be applied to Softmax as well
- Softmax complexity is $\mathcal{O}(|V|)$

![Word2Vec_Softmax](images/Word2Vec_Softmax.jpg)

## Hierarchical Softmax (H-Softmax)

- Decompose the flat hierarchy into a binary tree
- Form a hierarchical description of a word as a sequence of $\mathcal{O}(log_2 |V|)$ decision and there by reducing the computing complexity of Softmax
  - $\mathcal{O}(|V|) \rightarrow \mathcal{O}(log_2(|V|))$
- Lay the words in a tree-based hierarchy - words as leaves
- Binary tree with $|V| - 1$ nodes for $left (0)$ and $right (1)$ traversal
- Every leave represents the probability of the word
- Path length of a Balanced Tree is $log_2(|V|)$.
  - If the $|V| = \texttt{1 million words}$, then the $length = \texttt{19.9 bits/word}$
- Constructing an _Huffman encoded-tree_ would help frequent words to have short unique binary codes
- Learn to take these probabilistic decisions instead of directly predicting each word's probability $\textbf{[Bengio:2003:NPL:944919.944966]}$
- Every intermediate node denotes the relative probabilities of its child nodes
- The path to reach every leaf (word) is unique
- H-Softmax in many cases increases the prediction speed by more than $50X$ times

### Hierarchical Tree Structure

![H_Softmax_Hierarchical_Tree_Structure](images/H_Softmax_Hierarchical_Tree_Structure.jpg)

### Advantages

- Decomposes the flat hierarchy into a binary tree
- The path to reach every leaf (word) is unique
- Lays the words in a tree-based hierarchy - words as leaves
- Binary tree with $|V| - 1$ nodes for left and right traversal
- Every intermediate node denotes the relative probabilities of its child nodes
- Every leaf represents the normalized probability of the word
- Each node is indexed by a bit vector corresponding to the path from the root to the node
  - Append $1$ or $0$ according to whether the $left$ or $right$ branch of a decision node
- Normalized values for the words are calcualted without finding the probability for every word
- The entire vocabulary is partitioned into classes
- ANN learns to take these probabilistic decisions instead of directly predicting each word's probability [[33]](References.ipynb)
- Forms a hiearchical description of a word as a sequence of $\mathcal{O}(log_2 |V|)$ decisions
- Reduces the computing complexity of Softmax - $\mathcal{O}(V) \rightarrow \mathcal{O}(log_2(|V|))$
- Path length of a Balanced Tree is $log_2(|V|)$.
  - If the $|V| = \texttt{1 million words}$, then the $length = \texttt{19.9 bits/word}$
- A Balanced Binary Tree should provide an exponential speed-up, on the order of $\frac{|V|}{log_2(|V|)}$
- Constructing an __Huffman encoded-tree__ would help frequent words to have short unique binary codes
- H-Softmax in many cases increases the prediction speed by more than $50X$ times

### Limitations

- Separate training is required for phrases
- Embeddings are learned based n a small local window surrounding words - good and bad share the almost the same embedding
- Does not address polysemy
- Does not use frequencies of term co-occurences

### Balanced Binary Tree

|  Softmax_Optimization_Balanced_Binary_Tree_1 | Softmax_Optimization_Balanced_Binary_Tree_2 |
| --- | --- |
| ![Softmax_Optimization_Balanced_Binary_Tree_1](images/Softmax_Optimization_Balanced_Binary_Tree_1.jpg) | ![Softmax_Optimization_Balanced_Binary_Tree_2](images/Softmax_Optimization_Balanced_Binary_Tree_2.jpg) |

### Mapping Output Layer to Softmax

- The output of H-Softmax changed the structure of the output
  - It doesn't match with the Neural Network's Output Layer

![Word2vec_WIth_Hierarchical_Softmax](images/Word2vec_WIth_Hierarchical_Softmax.jpg)

### Updating Weights using Hierarchical Softmax

- This is the core of recent NLP trends
  - Read the [papers](#Optimization_Readings) suggested by lecturer to understand more on the new teminologies used in this section
  - Watch videos and read papers multiple times to understand the concept

| H_Softmax_Updating_Weights_1 | H_Softmax_Updating_Weights_2 | H_Softmax_Updating_Weights_3 |
|----|----|---|
| ![H_Softmax_Updating_Weights_1](images/H_Softmax_Updating_Weights_1.jpg) | ![H_Softmax_Updating_Weights_2](images/H_Softmax_Updating_Weights_2.jpg) | ![H_Softmax_Updating_Weights_3](images/H_Softmax_Updating_Weights_3.jpg) |

## Word2Vec Results

- Read the code given in the [link](https://github.com/dav/word2vec) to understand how Word2Vec is implemented in C, it is a Google Open Source
- You can download, compile (make) and can use the scripts to run it

![Word2Vec_Results](images/Word2Vec_Results.jpg)

### GloVe

- Word2Vec is from Google
- GloVE is from Stanford University
- Both available in various word vector sizes - 50, 100, 30, ...
  - Readymade Word Vectors available for public use
- We may use the Word Embeddings(Word Vectors) from either of these (if in case we do not want to create our own Word Embeddings from our Corpus, which is a tedious process normally)

## Study Links

<a id='Optimization_Readings'></a>

- Papers used by Lecturer for preparing the session (Word2Vec in ANLP course). Read it for further understanding about Word2Vec
  1. [Yoshua Bengio et all. "A Neural Probabilitis Language Model"](https://dl.acm.org/citation.cfm?id=94419.944966)
    - __*Must Read*__, A Very Important Paper, available publicly
  2. [Tomas Mikolov et al. "Distributed Representations of Words and Phrases and Their Compositionality"](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
    - <https://dl.acm.org/citation.cfm?id=2999959>
    - __*Must Read*__, it talks about Word2Vec
  3. [Andrily Mnih and Geoffrey Hinton. "A Scalable Hierarchical Distributed Language Model"](https://dl.acm.org/citation.cfm?id=2981780.2981915)
    - Talks about the
      - Hierarchical Distributed Model
      - How we can place the words in the binary trees in an hierarchical fashion
  4. Jeffrey A. Dean Tomas Mikolov Kai ChenGregory S. Crrado.
    - _U.S. pat. US9037464B1. May 2015_
    - This is the original patent filed by Google
    - Gives information about how these processes completed in finding the word vectors
  5. Lecture 7105 from [NPTEL on Deep Learning](https://nptel.ac.in/courses/106/106/106106184/)
    - Especially on the Hierarchical Softmax
      - Briefly explains the theory of the Hierarchical Model, a good reference in understanding the material (material of this course)

![Word2Vec_Study_References](images/Word2Vec_Study_References.jpg)